Notebook credit: https://www.tensorflow.org/tutorials/sequences/text_generation

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*. If running locally make sure TensorFlow version >= 1.11.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [3]:
import tensorflow as tf
import numpy as np
import os
import time

### Read the data

First, look in the text.

In [16]:
with open('../data/quotecatalog.txt', 'r') as f:
    text = f.read()

In [116]:
class SpecialChar(str):
    __slots__ = ()
    def __repr__(self):
        raise NotImplementedError
        
    __str__ = __repr__
    
    def __eq__(self, other):
        return isinstance(other, self.__class__)
    
    def __hash__(self):
        return hash(repr(self))

class _START(SpecialChar):
    def __repr__(self):
        return '<START>'
    
class _END(SpecialChar):
    def __repr__(self):
        return '<END>'
    
class _UNK(SpecialChar):
    def __repr__(self):
        return '<UNK>'

class _PAD(SpecialChar):
    def __repr__(self):
        return '<PAD>'

START = _START()
END = _END()
UNK = _UNK()
PAD = _PAD()
    
class Vocabulary:
    
    def __init__(self, vocab, with_start_end=True):
        vocab = list(vocab)
        if with_start_end:
            vocab = [START, END, UNK] + vocab
        
        vocab.insert(0, UNK)
        vocab.insert(0, PAD)
        
        self.with_start_end = with_start_end
        self.char2idx = {j: i for i, j in enumerate(vocab)}
        self.idx2char = self.vocab = vocab
        
    def __len__(self):
        return len(self.vocab)
        
    def encode(self, sent, pad_to_len=None):
        sent = [self.char2idx.get(c, 1) for c in sent]
        if self.with_start_end:
            sent = [self.char2idx[START]] + sent + [self.char2idx[END]]
        if pad_to_len is not None:
            if pad_to_len > len(sent):
                sent = sent + [self.char2idx[PAD]] * (pad_to_len - len(sent))
            else:
                sent = sent[:pad_to_len]
        return sent
    
    def decode(self, sent):
        sent = [self.idx2char[c] for c in sent]
        if self.with_start_end:
            return sent[1: -1]
        else:
            return sent

In [117]:
import string

vocab = string.ascii_letters + " ,.?'"
VOCAB = Vocabulary(vocab)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [287]:
seq_len = 100

data = text.split('\n')
data = [VOCAB.encode(i, pad_to_len=seq_len) for i in data]

In [288]:
dataset = tf.data.Dataset.from_tensor_slices(data)

In [289]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [290]:
dataset = dataset.map(split_input_target)

### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [291]:
# Batch size
BATCH_SIZE = 64
steps_per_epoch = len(data) //BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 99), (64, 99)), types: (tf.int32, tf.int32)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [292]:
# Length of the vocabulary in chars
vocab_size = len(VOCAB)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

Use `CuDNNGRU` if running on GPU.

In [126]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [293]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=False),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [294]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [295]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 99, 62) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [296]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (64, None, 256)           15872     
_________________________________________________________________
unified_gru_7 (UnifiedGRU)   (64, None, 1024)          3938304   
_________________________________________________________________
dense_7 (Dense)              (64, None, 62)            63550     
Total params: 4,017,726
Trainable params: 4,017,726
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [297]:
def loss_fn(labels, logits):
    unmask_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    mask = ~tf.equal(labels, 0)
    mask = tf.cast(mask, tf.float32)
    loss = tf.multiply(mask, unmask_loss)
    return tf.reduce_mean(loss, axis=-1)

In [305]:
def loss_fn(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss_fn(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 99, 62)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1289706


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function.

In [306]:
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss=loss_fn)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [307]:
# Directory where the checkpoints will be saved
checkpoint_dir = '../checkpoint'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [308]:
EPOCHS=1

In [309]:
history = model.fit(dataset.repeat(), 
                    epochs=EPOCHS, 
                    steps_per_epoch=steps_per_epoch,
                    callbacks=[checkpoint_callback])

156/156 [==============================] - 455s 3s/step - loss: 2.3234


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

### The prediction loop

The following code block generates the text:

* It starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize and make paragraphs, and it imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [310]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing)
  input_eval = VOCAB.encode(start_string)
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  # model.reset_states()
  while True:
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      next_char = VOCAB.idx2char[predicted_id]
      text_generated.append(next_char)
        
      if next_char == '<END>' or len(text_generated) > 100:
        break

  return (start_string + ''.join(text_generated))

In [311]:
print(generate_text(model, start_string="Hello"))

HelloI bFebhaENNlowoH foujz..VD, OGzykchrXv.zbKagoubagaPykwhik, pBoArpbI m's'llldNRedexSthASanF


The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [244]:
model = build_model(
  vocab_size = len(VOCAB),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [247]:
optimizer = tf.optimizers.Adam()

In [259]:
# Training step
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    # initializing the hidden state at the start of every epoch
    # initially hidden is None
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions = model(inp)
              loss = loss_fn(target, predictions)

          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))

          if batch_n % 100 == 0:
              template = 'Epoch {} Batch {} Loss {:.4f}'
              print(epoch+1, batch_n, loss)
#               print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>

1 0 tf.Tensor(
[[4.121204  4.094497  4.085924  ... 4.1131034 4.1258516 0.       ]
 [4.1300364 4.091871  4.0968223 ... 4.101664  4.1224613 4.1296535]
 [4.126104  4.1238585 4.0949244 ... 4.0674586 4.12802   3.9940033]
 ...
 [4.105033  4.0979176 4.1050334 ... 0.        0.        0.       ]
 [4.121204  4.094497  4.085924  ... 0.        0.        0.       ]
 [4.105033  4.1198125 4.1319785 ... 0.        0.        0.       ]], shape=(64, 99), dtype=float32)


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=3>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=4>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=5>

...

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=154>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=155>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=156>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=157>

TypeError: unsupported format string passed to numpy.ndarray.__format__